#Simulating Users

### Possible Topics
* Setup:
    * Me (data science type, lots of for loops, clever in python)
    * Them (devs, specs, dbs, not inference, tests if you are lucky)
    * testing is a place we can all communicating
* A/B Testing (notion vs truth)
    * Diagram of simple vs Bing
* Problem statement (it's one thing to to execute a plan, it's another to achieve robust/trustworthy exp)
* Design vs Execution vs Scorecard
    * Design can add complexity
    * Execution is not analysis
    * Scorecard is a "view" not the truth
    * Parallel views (what someone did versus what we logged)
* Execution to Scorecard phase:
    * Empirically has had problems
    * Many are just incorrect execution
    * Some are incorrect aggregation
    * Most were fixable with unit testing
* Scaling and testing:
    * Don't need actual experiments...we can simulate
* Simulation
    * Humans are suprisingly simple when it comes to log lines
    * Artifact of what we log to some extent
    * Can produce surprisingly accurate aggregate results
* Simple flow:
    1. Human visits
    2. Human has choice (often influenced by treatment...you hope)
    3. Human makes choice
    4. (Optional) Human repeats 2 and 3 additional times
    * Key point: Logs are *generated* by a process
* Abstract the process:
    1. Present a choice (probability distribution)
    2. Draw from that distribution
    3. Given the draw, present a second choice (another probability distribution, possibly different)
    4. Draw again
    5. Repeat
    * Simple process but it captures the essence of the experiment
    * The layering of draws and choice of distributions inject flexibility and complexity
* Example:
    * Simple click stream of an offer
    * Provides a way for the devs to interact with the logs in multiple ways
    * Easy to write tests for summary stats
* Find the "unknown" parameters
    * Simulated logs are noisy instaniations of your supplied parameters
    * In other words, you put a number into the function and it spit out a ton of hand wavey examples
    * Your task (well, the dev's task) is to recover that parameter (within reason)
    * This is the foundation of the unit test framework
* Example test:
    * CTR difference between two groups
    * How do I deal with "random" data in an assert?
    * `Almost Equal`?
* Dealing with random data:
    * Seed (although this will not enable robustness)
    * p-value (expensive since I need to test the function ~1000 times)
    * Comparisons (balance between the two...i.e. do I always manage to find the more true one)
    * I tend to land on the seed method when starting, migrate to p-value for robustness, and do comparisons when devs complain my tests take too long (doesn't happen that often)
* End to end test for a conversion
    * what do I need to test?
        * Log integrity (are there illegal values?) Instrumentation problems (THIS IS A HUGE DEAL)
        * Roll ups (you'd be surprised how broken this is)
        * Reverse A/A (you are doing this right?)
        * Comparisons (This is sort of the important part)
* What else can I do?
    * Simulate hypotheses
    * Be really brute force about sample size calculation
    * Practice scaling
    * How do bugs or problem "trickle down"